In [16]:
import pyomo as pyo
from pyomo.environ import *
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
from shift_and_coverage import create_shifts, is_shift_covering_period

In [80]:
employee_count = 13
employee_ids = ['emp{}'.format(i) for i in range(employee_count)]
employee_ids

['emp0',
 'emp1',
 'emp2',
 'emp3',
 'emp4',
 'emp5',
 'emp6',
 'emp7',
 'emp8',
 'emp9',
 'emp10',
 'emp11',
 'emp12']

### Set of days in the planning horizon & Time periods
**Intervals_in_horizon is the Set of time periods 1....48 intervals in 30 min**
**days in the horizon are the Set of days in the planning horizon**

In [81]:
weeks_in_horizon = 1 #weeks
horizon_lenght = 1 * weeks_in_horizon 
period_length = 60
day_length = 24 * 60 // period_length
intervals_in_horizon = list(range(horizon_lenght * day_length))

days_in_horizon = list(range(horizon_lenght))
intervals_in_horizon

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23]

### Create demand for each time period

In [82]:
demand = {
0: 4,
 1: 3,
 2: 3,
 3: 2 ,
 4: 1,
 5: 1,
 6: 1,
 8: 1,
 9: 1,
 10: 2,
 11: 2,
 12: 2,
 13: 2,
 14: 1,
 15: 1,
 16: 2,
 17: 2,
 18: 2,
 19: 2,
 20: 2,
 21: 2,
 22: 3,
 23: 4,
}



### Shift creation, create possible shifts with min and max duration

In [83]:
#shifts =create_shifts(60)
shifts = [(6,10),(14,8),(22,10)]


### Pyomo Model

In [89]:
# Define the Pyomo model
model = ConcreteModel()

# Sets
model.E = Set(initialize=employee_ids)  # Set of employee IDs
model.T = Set(initialize=intervals_in_horizon)  # Set of time periods
model.S = Set(initialize=shifts)  # Set of possible shifts

# Variables
model.x = Var(model.E, model.S, domain=Binary) 

# Constraints
def demand_constraint_rule(model, time):
    if time in demand:
        return sum(model.x[employee, shift] for employee in model.E for shift in model.S if is_shift_covering_period(shift, time, day_length)) >= demand[time]
    else:
        # If demand for this period is not defined, no constraint is needed
        return Constraint.Skip

model.demand_constraint = Constraint(model.T, rule=demand_constraint_rule)

def exclusive_shifts_constraint(model, employee, time):
    assigned_shifts = [shift for shift in model.S if (employee, shift) in model.x and value(model.x[employee, shift]) > 0.5 and shift[0] <= time < shift[0] + shift[1]]
    
    if len(assigned_shifts) <= 1:
        return Constraint.Skip  # Skip constraint if employee is assigned to 0 or 1 shift

    assigned_shift_starts = [shift[0] for shift in assigned_shifts]
    assigned_shift_starts.sort()

    for i in range(len(assigned_shift_starts) - 1):
        if assigned_shift_starts[i] == assigned_shift_starts[i + 1]:  # Check if consecutive shifts start on the same day
            return sum(model.x[employee, shift] for shift in assigned_shifts) <= 1

    return Constraint.Skip  # No consecutive shifts starting on the same day

model.exclusive_shifts_constraint = Constraint(model.E, model.T, rule=exclusive_shifts_constraint)



model.obj = Objective(expr=1, sense=minimize)

# Solve the model
solver = SolverFactory('gurobi')
results = solver.solve(model)

# Print results
if str(results.solver.termination_condition) == "optimal":
    print("Solution found:")
    for employee in model.E:
        for shift in model.S:
            if value(model.x[employee, shift]) > 0.5:
                print(f"Assign {employee} to shift starting at {shift[0]} with duration {shift[1]}")
else:
    print("No feasible solution found.")


ERROR: Rule failed when generating expression for Constraint demand_constraint
with index 7: KeyError: 7
ERROR: Constructing component 'demand_constraint' from data=None failed:
KeyError: 7


KeyError: 7

In [88]:
 sum(model.x[employee, shift] for shift in model.S if is_shift_covering_period(shift, time, day_length))

NameError: name 'time' is not defined

In [73]:
def exclusive_shifts_constraint(model, employee, time):
    # Employee 'e' cannot work two shifts simultaneously
    return sum(model.x[employee, shift] for shift in model.S if is_shift_covering_period(shift, time, day_length)) <= 1

model.exclusive_shifts_constraint = Constraint(model.E, model.T, rule=exclusive_shifts_constraint)


exclusive_shifts_constraint (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>). This
is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


RuntimeError: Cannot add component 'exclusive_shifts_constraint_index' (type <class 'pyomo.core.base.set.SetProduct_OrderedSet'>) to block 'unknown': a component by that name (type <class 'pyomo.core.base.set.SetProduct_OrderedSet'>) is already defined.

In [74]:
exclusive_shifts_constraint(model, employee, time)

NameError: name 'time' is not defined

In [75]:
value(sum(model.x[employee, shift] for shift in model.S if is_shift_covering_period(shift, 6, day_length)) <= 1)

True

In [76]:
is_shift_covering_period(shift,9, day_length)

False

In [77]:
for  shift in model.S: print(shift) 

(6, 10)
(14, 8)
(22, 10)


In [78]:
model.x.display()

x : Size=21, Index=x_index
    Key              : Lower : Value : Upper : Fixed : Stale : Domain
     ('emp0', 6, 10) :     0 :   0.0 :     1 : False : False : Binary
     ('emp0', 14, 8) :     0 :   0.0 :     1 : False : False : Binary
    ('emp0', 22, 10) :     0 :   1.0 :     1 : False : False : Binary
     ('emp1', 6, 10) :     0 :   0.0 :     1 : False : False : Binary
     ('emp1', 14, 8) :     0 :   1.0 :     1 : False : False : Binary
    ('emp1', 22, 10) :     0 :   0.0 :     1 : False : False : Binary
     ('emp2', 6, 10) :     0 :   1.0 :     1 : False : False : Binary
     ('emp2', 14, 8) :     0 :   0.0 :     1 : False : False : Binary
    ('emp2', 22, 10) :     0 :   0.0 :     1 : False : False : Binary
     ('emp3', 6, 10) :     0 :   1.0 :     1 : False : False : Binary
     ('emp3', 14, 8) :     0 :   0.0 :     1 : False : False : Binary
    ('emp3', 22, 10) :     0 :   0.0 :     1 : False : False : Binary
     ('emp4', 6, 10) :     0 :   0.0 :     1 : False : False : 